<a href="https://colab.research.google.com/github/pravallika89ch/alignn/blob/main/internship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()


✨🍰✨ Everything looks OK!


In [3]:
import os
os.environ["MPLBACKEND"] = "Agg"


In [4]:
!conda install -y numpy=1.23.5 pandas=1.5.3 python=3.10
!pip install jarvis-tools alignn torchdata dgl==1.1.1



Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | failed

SpecsConfigurationConflictError: Requested specs conflict with configured specs.
  requested specs: 
    - numpy=1.23.5
    - pandas=1.5.3
    - python=3.10
  pinned specs: 
    - cuda-version=12
    - python=3.11
    - python_abi=3.11[build=*cp311*]
Use 'conda config --show-sources' to look for 'pinned_specs' and 'track_features'
configuration parameters.  Pinned specs may also be defined in the file
/usr/local/conda-meta/pinned.




In [17]:
import pandas as pd
import json
import os
from jarvis.core.atoms import Atoms

# Set your folder paths
csv_path = "/content/drive/MyDrive/Internship/tmdc-co2-meoh/dataset/opentmdc/co2.vaspfiles/id_prop.csv"
vasp_folder = "/content/drive/MyDrive/Internship/tmdc-co2-meoh/dataset/opentmdc/co2.vaspfiles"
output_json = "co2_dataset.json"

# Load CSV
df = pd.read_csv(csv_path)

# Prepare dataset list
dataset = []
for i, row in df.iterrows():
    vasp_file = os.path.join(vasp_folder, row['id'])
    atoms = Atoms.from_poscar(vasp_file)
    dataset.append({
        "structure": atoms.to_dict(),
        "target": float(row['adsorption_energy (eV)']),
        "jid": row['id']
    })

# Save JSON
with open(output_json, "w") as f:
    json.dump(dataset, f, indent=2)

print("Dataset JSON created:", output_json)

Dataset JSON created: co2_dataset.json


In [18]:
import json

# Load the original file
with open("co2_dataset.json", "r") as f:
    data = json.load(f)

# Fix structure ➝ atoms
for entry in data:
    entry["atoms"] = entry.pop("structure")

# Save it back
with open("co2_dataset_fixed.json", "w") as f:
    json.dump(data, f, indent=2)

print("Fixed file written as co2_dataset_fixed.json")

Fixed file written as co2_dataset_fixed.json


In [19]:
import json, random

with open("co2_dataset_fixed.json") as f:
    data = json.load(f)

# Shuffle for randomness
random.shuffle(data)

# Split
train, val, test = data[:30], data[30:33], data[33:36]

# Save
with open("train.json", "w") as f: json.dump(train, f, indent=2)
with open("val.json", "w") as f: json.dump(val, f, indent=2)
with open("test.json", "w") as f: json.dump(test, f, indent=2)

print("✅ train/val/test JSON files created.")

✅ train/val/test JSON files created.


In [20]:
import json

with open("train.json", "r") as f:
    train_data = json.load(f)

print(f"✅ Total training samples: {len(train_data)}")


✅ Total training samples: 30


In [21]:
config = {
    "filename": "co2_dataset_fixed.json",
    "dataset": "user_data",
    "use_lmdb": False,
    "target": "target",
    "id_tag": "jid",
    "atom_features": "cgcnn",
    "neighbor_strategy": "k-nearest",
    "dtype": "float32",
    "random_seed": 7,
    "n_train": 27,
    "n_val": 6,
    "n_test": 3,  # total: 36
    "epochs": 100,  # more chances for val improvement
    "batch_size": 8,  # smaller batch = more updates
    "learning_rate": 0.0003,  # slower learning for stability
    "weight_decay": 1e-5,
    "warmup_steps": 1000,
    "criterion": "mse",
    "optimizer": "adamw",
    "scheduler": "onecycle",
    "n_early_stopping": 10,  # stop if val loss stagnates
    "log_tensorboard": False,
    "standard_scalar_and_pca": True,  # normalize target values!
    "write_checkpoint": True,
    "write_predictions": True,
    "store_outputs": True,
    "progress": True,
    "compute_line_graph": True,
    "cutoff": 6.0,
    "cutoff_extra": 2.0,
    "max_neighbors": 12,
    "keep_data_order": True,
    "model": {
        "name": "alignn",
        "alignn_layers": 2,
        "gcn_layers": 2,
        "atom_input_features": 92,
        "edge_input_features": 80,
        "triplet_input_features": 40,
        "embedding_features": 32,
        "hidden_features": 128,
        "output_features": 1,
        "link": "identity",
        "zero_inflated": False,
        "classification": False,
        "num_classes": 2,
        "extra_features": 0
    }
}


In [22]:
import json

with open("configfile.json", "w") as f:
    json.dump(config, f, indent=2)

print("✅ Config saved successfully!")


✅ Config saved successfully!


In [11]:
%env MPLBACKEND=Agg


env: MPLBACKEND=Agg


In [23]:
!git clone https://github.com/usnistgov/alignn.git

fatal: destination path 'alignn' already exists and is not an empty directory.


In [13]:
import os
os.environ["MPLBACKEND"] = "Agg"
import matplotlib
matplotlib.use("Agg")


In [24]:
import json

with open("co2_dataset_fixed.json") as f:
    data = json.load(f)

print(f"📦 Loaded {len(data)} entries")
print("🔑 First record keys:", data[0].keys() if data else "Empty dataset")


📦 Loaded 36 entries
🔑 First record keys: dict_keys(['target', 'jid', 'atoms'])


In [26]:
!python /content/alignn/alignn/train.py /content/configfile.json



✅ alignn loaded from: /content/alignn/alignn/__init__.py
fatal: not a git repository (or any of the parent directories): .git
/content/alignn/alignn/train.py:82: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print("config:", config.dict())
config: {'version': 'NA', 'dataset': 'user_data', 'target': 'target', 'atom_features': 'cgcnn', 'neighbor_strategy': 'k-nearest', 'id_tag': 'jid', 'dtype': 'float32', 'random_seed': 7, 'classification_threshold': None, 'n_val': 6, 'n_test': 3, 'n_train': 27, 'train_ratio': 0.8, 'val_ratio': 0.1, 'test_ratio': 0.1, 'target_multiplication_factor': None, 'epochs': 100, 'batch_size': 8, 'weight_decay': 1e-05, 'learning_rate': 0.0003, 'filename': 'co2_dataset_fixed.json', 'warmup_steps': 1000, 'criterion': 'mse', 'optimizer': 'adamw', 'scheduler': 'onecycle', 'pin_memory': False, 'save_